In [1]:
from biocrnpyler.components.dna import DNAassembly, ActivatablePromoter
from biocrnpyler.mixtures import EnergyTxTlExtract, SimpleTxTlExtract, ExpressionExtract, TxTlExtract, ExpressionDilutionMixture, TxTlDilutionMixture
from biocrnpyler import Compartment
from biocrnpyler import Species, Component
import tempfile
import libsbml

activator = Species("T7RNAP", material_type = "small_molecule")
hill_parameters = {"k":1.0, "n":4, "K":20, "kleak":0.0001}
P_activatable = ActivatablePromoter("P_activtable", activator = activator,
                                    leak = False, parameters = hill_parameters)
activatable_assembly = DNAassembly("activatable_assembly",
                                   promoter = P_activatable, rbs = "rbs",
                                   initial_concentration = 1*10**-3, 
                                   protein= "GFP")
# E = TxTlDilutionMixture(components=[activatable_assembly],
#                       parameter_file = "default_parameters.txt")
E = EnergyTxTlExtract(components=[activatable_assembly],
                      parameter_file = "examples/Specialized Tutorials/txtl_toolbox_parameters.txt")
CRN1 = E.compile_crn() #comple CRN
new_compartment = Compartment("Internal_0")
E.compartment = new_compartment
CRN2 = E.compile_crn() #compile CRN
CRN1.write_sbml_file('CRN1.xml')
CRN2.write_sbml_file('CRN2.xml')

model1 = libsbml.readSBMLFromFile('CRN1.xml').getModel()
model2 = libsbml.readSBMLFromFile('CRN2.xml').getModel()

all_species1 = model1.getListOfSpecies()
all_species2 = model2.getListOfSpecies()

# check that all species are in the new compartment
problem_species = []
for s1, s2 in zip(all_species1, all_species2):
    # if s1.getName() != s2.getName():
        # print("%s does not have same name as %s" %(s1.name, s2.name))
    compartment1 = s1.getCompartment()
    compartment2 = s2.getCompartment()
    if compartment1 == compartment2:
        print("For %s original compartment is %s and new is %s" %(s1.name, compartment1, compartment2))

c:\Users\ayush\Box\Research\bioCRNpyler\biocrnpyler\core\parameter.py:507: UserWarning: parameter file contains no unit column! Please add a column named ['unit', 'units'].
  warn(f"parameter file contains no {accepted_name} column! Please add a "


In [9]:
CRN1.reactions

[dna[activatable_assembly] --> dna[activatable_assembly]+rna[activatable_assembly],
 rna[activatable_assembly]+protein[Ribo] <--> complex[protein[Ribo]:rna[activatable_assembly]],
 4metabolite[NTPs]+metabolite[amino_acids]+complex[protein[Ribo]:rna[activatable_assembly]] --> 4metabolite[NTPs]+metabolite[amino_acids]+rna[activatable_assembly]+protein[Ribo]+protein[GFP],
 4metabolite[NTPs]+metabolite[amino_acids]+complex[protein[Ribo]:rna[activatable_assembly]] --> complex[protein[Ribo]:rna[activatable_assembly]]+4metabolite[NDPs],
 metabolite[Fuel_3PGA]+metabolite[NDPs] --> metabolite[NTPs],
 metabolite[NTPs] --> metabolite[NDPs],
 rna[activatable_assembly]+protein[RNAase] <--> complex[protein[RNAase]:rna[activatable_assembly]],
 complex[protein[RNAase]:rna[activatable_assembly]] --> protein[RNAase],
 complex[protein[Ribo]:rna[activatable_assembly]]+protein[RNAase] <--> complex[complex[protein[Ribo]:rna[activatable_assembly]]:protein[RNAase]],
 complex[complex[protein[Ribo]:rna[activata

In [8]:
CRN2.reactions

[dna[activatable_assembly] --> dna[activatable_assembly]+rna[activatable_assembly],
 rna[activatable_assembly]+protein[Ribo] <--> complex[protein[Ribo]:rna[activatable_assembly]],
 4metabolite[NTPs]+metabolite[amino_acids]+complex[protein[Ribo]:rna[activatable_assembly]] --> 4metabolite[NTPs]+metabolite[amino_acids]+rna[activatable_assembly]+protein[Ribo]+protein[GFP],
 4metabolite[NTPs]+metabolite[amino_acids]+complex[protein[Ribo]:rna[activatable_assembly]] --> complex[protein[Ribo]:rna[activatable_assembly]]+4metabolite[NDPs],
 metabolite[Fuel_3PGA]+metabolite[NDPs] --> metabolite[NTPs],
 metabolite[NTPs] --> metabolite[NDPs],
 rna[activatable_assembly]+protein[RNAase] <--> complex[protein[RNAase]:rna[activatable_assembly]],
 complex[protein[RNAase]:rna[activatable_assembly]] --> protein[RNAase],
 complex[protein[Ribo]:rna[activatable_assembly]]+protein[RNAase] <--> complex[complex[protein[Ribo]:rna[activatable_assembly]]:protein[RNAase]],
 complex[complex[protein[Ribo]:rna[activata

In [47]:
CRN2.species[0]

complex_protein_Ribo_rna_activatable_assembly_Internal_0_

In [46]:
CRN2.species[3]

complex_protein_Ribo_rna_activatable_assembly_Internal_0_

In [45]:
CRN2.species[4]

complex_protein_Ribo_rna_activatable_assembly_Internal_0_

In [1]:
from biocrnpyler.components import Enzyme
from biocrnpyler.mechanisms import BasicCatalysis
from biocrnpyler.core import Mixture, Compartment

default_parameters = {"kb":100, "ku":10, "kcat":1.}
enzyme = Enzyme("E", substrates = "S", products = "P")
mech_cat = BasicCatalysis()
default_mechanisms = {mech_cat.mechanism_type:mech_cat}

M = Mixture("Catalysis", components = [enzyme],
            parameters = default_parameters, mechanisms = default_mechanisms)
CRN_1 = M.compile_crn()
CRN_1.write_sbml_file("CRN_1.xml")

new_compartment = Compartment("Internal_1")
M.compartment = new_compartment 
# assert M.compartment.name == "Internal_1" # works

# M.components[0].compartment = new_compartment
# assert M.components[0].compartment.name == "Internal_1" # works

CRN_2 = M.compile_crn()
CRN_2.write_sbml_file("CRN_2.xml") # this ends up being in default still, all species are in default compartment even though the component and mixture compartment was updated. 


# for s in CRN_2.species:
#     s.compartment = new_compartment
#     print(s) # prints species names appended with compartments (as expected) but updated CRN_3 sbml (see below) does not have these updated names.
#     assert s.compartment.name == "Internal_1" # works for all species

# CRN_2.write_sbml_file("CRN_3.xml") # this still does not update compartment for all species. Because we did not update the objects in mixture and compilation had already happened.

True

In [2]:
CRN_2.species

[protein_E_Internal_1, S_Internal_1, P_Internal_1]

In [3]:
CRN_2.reactions

[protein[E]+S --> protein[E]+P]

In [2]:
CRN_1

Species = protein_E, S, P
Reactions = [
	protein[E]+S --> protein[E]+P
]

In [34]:
M.components[0]

Enzyme: E

In [17]:
CRN_2.species

[protein_E, S, P]

In [4]:
from biocrnpyler.components.dna import DNAassembly, ActivatablePromoter
from biocrnpyler.mixtures import EnergyTxTlExtract, SimpleTxTlExtract, ExpressionExtract, TxTlExtract, ExpressionDilutionMixture, TxTlDilutionMixture
from biocrnpyler import Compartment
from biocrnpyler import Species, Component
import tempfile
import libsbml
import os
hill_parameters = {"k":1.0, "n":4, "K":20, "kleak":0.0001}
P_activatable = ActivatablePromoter("P_activtable", activator = "activator",
                                    parameters = hill_parameters)
activatable_assembly = DNAassembly("activatable_assembly",
                                promoter = P_activatable)
E = EnergyTxTlExtract(components=[activatable_assembly],
                        parameter_file="Specialized Tutorials/txtl_toolbox_parameters.txt")
CRN_1 = E.compile_crn()  # compile CRN

new_compartment = Compartment(name="Internal_0")
CRN_2 = E.compile_crn(compartment=new_compartment)  # compile CRN

assert len(CRN_1.species) == len(CRN_2.species)
assert len(CRN_1.reactions) == len(CRN_2.reactions) 

# Assert SBML compartments
with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
    CRN_1.write_sbml_file(tmp.name)
    model = libsbml.readSBMLFromFile(tmp.name).getModel()
    for sbml_species in model.getListOfSpecies():
        assert sbml_species.getCompartment() == "default"
os.remove(tmp.name)

with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
    CRN_2.write_sbml_file(tmp.name)
    model = libsbml.readSBMLFromFile(tmp.name).getModel()
    for sbml_species in model.getListOfSpecies():
        assert sbml_species.getCompartment() == "Internal_0"
os.remove(tmp.name)

C:\Users\ayush\Box\Research\bioCRNpyler\biocrnpyler\core\parameter.py:507: UserWarning: parameter file contains no unit column! Please add a column named ['unit', 'units'].
  warn(f"parameter file contains no {accepted_name} column! Please add a "


after adding the species is now [protein_RNAase_Internal_0, rna_activatable_assembly_Internal_0, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_]


Current species list is {dna_activatable_assembly_Internal_0: True, small_molecule_T7RNAP_Internal_0: True, rna_activatable_assembly_Internal_0: True, metabolite_NTPs_Internal_0: True, metabolite_amino_acids_Internal_0: True, protein_Ribo_Internal_0: True, protein_GFP_Internal_0: True, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, protein_RNAP_Internal_0: True, protein_RNAase_Internal_0: True, metabolite_Fuel_3PGA_Internal_0: True, metabolite_NDPs_Internal_0: True, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, complex_protein_Ribo_rna_activatable_assembly_Internal_0_: True, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_: True, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_: True} 


and x0 is [protein_RNAase_Internal_0, rna_activatable_assembly_Internal_0, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_]


Current species list is 

{dna_activatable_assembly_Internal_0: True, 
small_molecule_T7RNAP_Internal_0: True, rna_activatable_assembly_Internal_0: True, metabolite_NTPs_Internal_0: True, metabolite_amino_acids_Internal_0: True, protein_Ribo_Internal_0: True, protein_GFP_Internal_0: True, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, protein_RNAP_Internal_0: True, protein_RNAase_Internal_0: True, metabolite_Fuel_3PGA_Internal_0: True, metabolite_NDPs_Internal_0: True, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, complex_protein_Ribo_rna_activatable_assembly_Internal_0_: True, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_: True, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_: True}

x0 is {protein_RNAase_Internal_0: 0, rna_activatable_assembly_Internal_0: 0, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: 0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: 0, protein_Ribo_Internal_0: 0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_: 0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_: 0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_: 0}

Current species list is {dna_activatable_assembly_Internal_0: True, small_molecule_T7RNAP_Internal_0: True, rna_activatable_assembly_Internal_0: True, metabolite_NTPs_Internal_0: True, metabolite_amino_acids_Internal_0: True, protein_Ribo_Internal_0: True, protein_GFP_Internal_0: True, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, protein_RNAP_Internal_0: True, protein_RNAase_Internal_0: True, metabolite_Fuel_3PGA_Internal_0: True, metabolite_NDPs_Internal_0: True, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, complex_protein_Ribo_rna_activatable_assembly_Internal_0_: True, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_: True, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_: True, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_: True}


x0 is [protein_RNAase_Internal_0, rna_activatable_assembly_Internal_0, complex_protein_RNAase_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_Internal_0__protein_RNAase_Internal_0_Internal_0_, protein_RNAase_Internal_0, complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0_, protein_Ribo_Internal_0, complex_complex_protein_Ribo_Internal_0_rna_activatable_assembly_Internal_0__protein_RNAase_Internal_0_Internal_0_]



In [2]:
activatable_assembly.compartment

In [8]:
from biocrnpyler import Mixture
comp1 = Compartment(name="comp1")
comp2 = Compartment(name="comp2") 

s1 = Species("S1", compartment=comp1)
mixture_1 = Mixture(species=[s1])
crn_1 = mixture_1.compile_crn()
crn_1.write_sbml_file("test_compartment_setter_1.xml")
# change compartment
s1.compartment = comp2
mixture_2 = Mixture(species=[s1])
crn_2 = mixture_2.compile_crn()
assert s1.compartment.name == "comp2"

crn_2.write_sbml_file("test_compartment_setter_2.xml")
model = libsbml.readSBMLFromFile("test_compartment_setter_2.xml").getModel()
sbml_species = model.getSpecies(0)
assert sbml_species.getCompartment() == "comp2"

In [ ]:
from biocrnpyler.core import Compartment
from biocrnpyler.components import DNAassembly
from biocrnpyler.mixtures import TxTlExtract
import os
import tempfile
import libsbml
# Create a component
default_compartment = Compartment(name="default")
dna = DNAassembly("dna", compartment=default_compartment)
default_compartment2 = Compartment(name="default")
E = TxTlExtract(components=[dna], 
                parameter_file="../../examples/default_parameters.txt", compartment=default_compartment2)
assert E.rnap.species.compartment.name == "default"
# new_compartment = Compartment(name="Internal_0")
# E.compartment = new_compartment
# E.rnap.compartment = new_compartment
# E.rnap.species.compartment = new_compartment
# CRN = E.compile_crn()  # comple CRN
# assert E.rnap.species.compartment.name == "Internal_0"
# with tempfile.NamedTemporaryFile(suffix='.xml', delete=False) as tmp:
#     CRN.write_sbml_file(tmp.name)
#     model = libsbml.readSBMLFromFile(tmp.name).getModel()
#     sbml_species = model.getSpecies(0)
#     assert sbml_species.getCompartment() == "Internal_0"
# os.remove(tmp.name)

In [ ]:
E.compartment.name